<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Импортируем все необходимые инструменты:

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Прочитаем датасет:

In [54]:
df = pd.read_csv('/datasets/insurance.csv')
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


Данный датасет не имеет каких либо серьезных ошибок, так же не имеет смысла преобразовывать, чтобы выровнять вес признаков, так как в постановке задачи не нужно добиваться максимального качества, а лишь сравнить оригинальный датасет и преобразованный по метрике R2.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится. 

**Обоснование:**
Возьмем формулу весов и докажем, что итоговый результат не изменится:

$
w = (X^T X)^{-1} X^T y
$

$
X' = XP
$

$
w' = ((XP)^T(XP))^{-1}(XP)^Ty
$

$
w' = (X^T(P^TXP))^{-1}X^TP^Ty
$

$
w' = (X^TXP))^{-1}(P^T)^{-1}P^TX^Ty
$

$$
E = AA^{-1}
$$

$
w' = P^{-1}(X^T X)^{-1} X^T y
$

$$
w = (X^T X)^{-1} X^T y
$$

$
w' = P^{-1}w
$

Далее сделаем то же самое с формулой предсказаний:


$
a' = X'w'
$

$
a' = XPP^{-1}w = Xw = a
$


## Алгоритм преобразования

**Алгоритм**

$
X' = XP
$

**Обоснование**

Использование данного алгоритма обосновано в пункте 2


## Проверка алгоритма

Создадим матрицу и проверим её на обратимость:

In [60]:
np.random.seed(12345)
P = np.random.normal(size = (4, 4))
np.linalg.inv(P) 

array([[-1.31136747,  0.3921804 ,  0.18868055, -0.67088287],
       [ 1.75872714,  0.14106138, -0.17773045,  0.79787127],
       [-0.41702659, -0.22854768,  0.3550602 ,  0.33039819],
       [ 0.58912996,  0.19073027, -0.5545481 ,  0.6259302 ]])

Далее разделим датасет на выборки:

In [56]:
target = df['Страховые выплаты']
features = df.drop('Страховые выплаты', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Умножим выборку с признаками на матрицу и так же разделим на выборки для обучения модели:

In [57]:
features_1 = features.dot(P)
features_1
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target, test_size=0.25, random_state=12345)

Далее посчитаем метрику R2 на признаках до умножения на матрицу:

In [58]:
model_lr = LinearRegression()
model_lr.fit(features_train, target_train) 
predictions_LR = model_lr.predict(features_valid)
print("R2 =", r2_score(target_valid, predictions_LR))

R2 = 0.435227571270266


Теперь посчитаем метрику R2 на датасете уже преобразованном матрицей:

In [59]:
model_lr_1 = LinearRegression()
model_lr_1.fit(features_train_1, target_train_1) 
predictions_LR_1 = model_lr_1.predict(features_valid_1)
print("R2 =", r2_score(target_valid_1, predictions_LR_1))

R2 = 0.4352275712700221


**Вывод**
Как в доказательстве во второй части данного проекта так и на практике в четвертой умножение на обратимую матрицу не меняет качества предсказания модели. В обоих случаях получили метрику R2 равную 0.43